In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_landsat()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_h,wp_le,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,2.605590,13.643902,0.377924,0.413754,0.594708,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,3.459300,13.498761,0.381184,0.420529,0.597960,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,2.117474,14.987823,0.384444,0.427304,0.601212,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,4.636145,11.355851,0.387704,0.434079,0.604465,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,3.448946,10.943102,0.390963,0.440854,0.607717,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 24 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 non-null float64
wp_LST.night    1028 non-null float64
LW_IN.wp        1028 non-null float64
LW_IN.si        1028 non-null float64
LW_IN           1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_RNET"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.972026302876
CV Scores:  0.966090364096, 0.981395440963, 0.981144624359, 0.977471487488, 0.969910807526, 0.981257335623, 0.980886520111, 0.981861407596, 0.978991299699, 0.921253741301
OOB score: 0.971256496973
Feature Importances:
('sw_in', 0.29877820582945575)
('PET', 0.2524540659654953)
('wp_LST.day', 0.14596079419258257)
('wp_LST.night', 0.091478308898755481)
('air_temp', 0.076286615566341365)
('VPD', 0.048891848397527582)
('wp_evi', 0.029834814253632234)
('wp_lswi2', 0.018629254533967531)
('wp_ndvi', 0.013864963107228566)
('wind_speed', 0.012575766671730611)
('LW_IN.wp', 0.007983286525887999)
('precip', 0.0032620760573948441)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.973741175501
CV Scores:  0.973037763007, 0.981140430877, 0.97519585703, 0.980517312633, 0.972587902189, 0.980960415841, 0.980763576203, 0.984648132474, 0.979288283204, 0.929272081551
Feature Importances:
('sw_in', 0.17629077403149832)
('VPD', 0.10031280691021521)
('wp_evi', 0.087960983987877897)
('wp_LST.night', 0.087553162572414203)
('wp_LST.day', 0.087239424151564007)
('PET', 0.086193534002939054)
('LW_IN.wp', 0.082116923515685086)
('wp_lswi2', 0.076628734861153597)
('wind_speed', 0.07447627735483292)
('wp_ndvi', 0.069440494542707065)
('air_temp', 0.049010675908821695)
('precip', 0.022776208160290981)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.967587270769
CV Scores:  0.960316685258, 0.979234901039, 0.986230101784, 0.953187777725, 0.97443973957, 0.970098360809, 0.973529786776, 0.984350907264, 0.974069790765, 0.920414656704


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 133.01241
Step #501, epoch #50, avg. loss: 3.53756
Step #1001, epoch #100, avg. loss: 1.35883
Step #1501, epoch #150, avg. loss: 1.00814
Step #2001, epoch #200, avg. loss: 0.67247
Step #2501, epoch #250, avg. loss: 0.48562
Step #3001, epoch #300, avg. loss: 0.38801
Step #3501, epoch #350, avg. loss: 0.33583
Step #4001, epoch #400, avg. loss: 0.33597
Step #4501, epoch #450, avg. loss: 0.26541
Step #1, avg. loss: 165.01688
Step #501, epoch #50, avg. loss: 3.55745
Step #1001, epoch #100, avg. loss: 1.42933
Step #1501, epoch #150, avg. loss: 1.07934
Step #2001, epoch #200, avg. loss: 0.81004
Step #2501, epoch #250, avg. loss: 0.62621
Step #3001, epoch #300, avg. loss: 0.49878
Step #3501, epoch #350, avg. loss: 0.40455
Step #4001, epoch #400, avg. loss: 0.31386
Step #4501, epoch #450, avg. loss: 0.31677
Step #1, avg. loss: 160.71883
Step #501, epoch #50, avg. loss: 3.35837
Step #1001, epoch #100, avg. loss: 1.16441
Step #1501, e

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x10e2af488>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)